
# Star Wars Action Figures — EDA (Edited)
**Goal:** Concise, efficient descriptive analysis of Star Wars action figure sales with clear sections and reusable helpers.
*Libraries:* pandas, numpy, seaborn, matplotlib (Python 3.13.5 compatible)


In [ ]:

# SECTION 0 — Imports & Style
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", context="talk", palette="deep")
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.grid"] = True
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [ ]:

# SECTION 1 — Load Data
CSV_PATH = "../data/starwars/starwars_mocloose_sales_202510061222.csv"  # Edit if needed

assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH, low_memory=False)

# Best-effort parse date-like columns
for c in df.columns:
    if any(k in c.lower() for k in ["date", "time", "timestamp"]):
        try:
            df[c] = pd.to_datetime(df[c], errors="coerce")
        except Exception:
            pass

print(f"Loaded: {CSV_PATH}\nRows: {len(df):,} • Cols: {df.shape[1]}")
df.head(10)


In [ ]:

# SECTION 2 — Schema & Missingness
print("Dtypes:")
print(df.dtypes)

schema = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "non_null": df.notna().sum(),
    "missing": df.isna().sum(),
})
schema["missing_pct"] = (schema["missing"] / len(df) * 100).round(2)

schema.sort_values("missing", ascending=False)


In [ ]:

# SECTION 3 — Filter Config & Helper
YEAR_MIN = int(df["year"].min()) if "year" in df.columns else None
YEAR_MAX = int(df["year"].max()) if "year" in df.columns else None

# Adjust as needed when exploring
flt_year_min, flt_year_max = YEAR_MIN, YEAR_MAX   # e.g., 2010, 2020
include_conditions = []                            # e.g., ["moc_figure"]
exclude_conditions = []                            # e.g., ["loose_figure"]
authenticity_keep = [0, 1]                         # choose among [0], [1], or [0,1]
q_low, q_high = 0.01, 0.99                         # trim price tails
USE_LOG = False                                    # add 'selling_price_log' if True

def apply_filters(dfin: pd.DataFrame) -> pd.DataFrame:
    d = dfin.copy()
    # Year
    if "year" in d.columns and flt_year_min is not None and flt_year_max is not None:
        d = d[(d["year"] >= flt_year_min) & (d["year"] <= flt_year_max)]
    # Authenticity
    if "authenticity_n" in d.columns and len(authenticity_keep):
        d = d[d["authenticity_n"].isin(authenticity_keep)]
    # Conditions include/exclude
    if "condition" in d.columns and include_conditions:
        d = d[d["condition"].isin(include_conditions)]
    if "condition" in d.columns and exclude_conditions:
        d = d[~d["condition"].isin(exclude_conditions)]
    # Price trim + optional log
    if "selling_price" in d.columns and len(d):
        lo, hi = d["selling_price"].quantile([q_low, q_high])
        d = d[(d["selling_price"] >= lo) & (d["selling_price"] <= hi)].copy()
        if USE_LOG:
            d["selling_price_log"] = np.log1p(d["selling_price"])
    return d

dfv = apply_filters(df)
print(f"Filtered rows: {len(dfv):,} / {len(df):,}")
dfv.head(5)


In [ ]:

# SECTION 4 — Descriptive Statistics (numeric & categorical)
desc_num = dfv.describe().T.round(2)
print("Numeric summary:")
display(desc_num)

cat_cols = dfv.select_dtypes(include=["object", "category"]).columns.tolist()
if cat_cols:
    desc_cat = dfv[cat_cols].describe(include=["object"]).T
    print("\nCategorical summary:")
    display(desc_cat)

# Quick cardinality & ranges
if cat_cols:
    print("\nCategorical cardinality (unique values):")
    display(pd.Series({c: dfv[c].nunique(dropna=True) for c in cat_cols}, name="unique").sort_values(ascending=False))

num_cols = dfv.select_dtypes(include=[np.number]).columns.tolist()
if num_cols:
    print("\nNumeric ranges (min/max):")
    display(pd.DataFrame({c: {"min": float(np.nanmin(dfv[c])), "max": float(np.nanmax(dfv[c]))} for c in num_cols}).T)


In [ ]:

# SECTION 5 — Correlation & Missingness
num_cols = [c for c in dfv.columns if pd.api.types.is_numeric_dtype(dfv[c])]
if len(num_cols) >= 2:
    corr = dfv[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Correlation (Pearson) — numeric")
    plt.show()
else:
    print("Not enough numeric columns for correlation.")

plt.figure(figsize=(10, 4))
sns.heatmap(dfv.isna(), cbar=False, cmap="mako")
plt.title("Missing Data Heatmap")
plt.xlabel("Columns"); plt.ylabel("Rows")
plt.show()


In [ ]:

# SECTION 6 — Distributions by Condition & Authenticity
num_cols = dfv.select_dtypes(include=[np.number]).columns

# By condition
for col in num_cols:
    plt.figure(figsize=(8, 4))
    sns.kdeplot(data=dfv, x=col, hue="condition", common_norm=False, fill=True, alpha=0.35, linewidth=1.2)
    plt.title(f"Distribution_by_Condition — {col}")
    plt.tight_layout(); plt.show()

# Combined Condition × Authenticity
if "authenticity_n" in dfv.columns:
    df_tmp = dfv.copy()
    df_tmp["condition_auth"] = df_tmp["condition"].astype(str) + " | auth=" + df_tmp["authenticity_n"].astype(str)
    for col in num_cols:
        plt.figure(figsize=(8, 4))
        sns.kdeplot(data=df_tmp, x=col, hue="condition_auth", common_norm=False, alpha=0.6, linewidth=1.2)
        plt.title(f"Distribution_by_Condition_and_Authenticity — {col}")
        plt.legend(title="Condition | Authenticity", bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.tight_layout(); plt.show()


In [ ]:

# SECTION 7 — Price by Condition / Character Type (Boxplots)
col = "selling_price_log" if USE_LOG and "selling_price_log" in dfv.columns else "selling_price"

# Condition
if "condition" in dfv.columns and col in dfv.columns:
    order = dfv.groupby("condition")[col].median().sort_values(ascending=False).index
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=dfv, x="condition", y=col, order=order, showfliers=False)
    plt.title(f"Price by condition — {'log' if col=='selling_price_log' else 'linear'}")
    plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

# Character type (top-N by count)
if "character_type" in dfv.columns and col in dfv.columns:
    topn = 12
    top_types = dfv["character_type"].value_counts().head(topn).index
    sub = dfv[dfv["character_type"].isin(top_types)]
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=sub, x="character_type", y=col, order=top_types, showfliers=False)
    plt.title(f"Price by character_type (Top {topn})")
    plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()


In [ ]:

# SECTION 8 — Authenticity vs Price
if "authenticity_n" in dfv.columns and "selling_price" in dfv.columns:
    col = "selling_price_log" if USE_LOG and "selling_price_log" in dfv.columns else "selling_price"
    plt.figure(figsize=(9, 4))
    sns.scatterplot(data=dfv, x="authenticity_n", y=col, alpha=0.25)
    sns.regplot(data=dfv, x="authenticity_n", y=col, scatter=False, lowess=True, color="black")
    plt.title("Authenticity vs price (scatter + LOWESS)")
    plt.tight_layout(); plt.show()
else:
    print("Need 'authenticity_n' and 'selling_price'.")


In [ ]:

# SECTION 9 — Yearly Trend (median price & count)
if "year" in dfv.columns and "selling_price" in dfv.columns:
    tmp = dfv.copy()
    tmp["year"] = pd.to_numeric(tmp["year"], errors="coerce")
    col = "selling_price_log" if USE_LOG and "selling_price_log" in tmp.columns else "selling_price"

    year_med = tmp.groupby("year")[col].median().dropna()
    year_cnt = tmp["year"].value_counts().sort_index()

    fig, ax1 = plt.subplots(figsize=(10, 5))
    sns.lineplot(x=year_med.index, y=year_med.values, marker="o", ax=ax1)
    ax1.set_xlabel("Year"); ax1.set_ylabel(f"Median {col}")
    ax1.set_title("Yearly median price (line) + listing count (bars)")

    ax2 = ax1.twinx()
    ax2.bar(year_cnt.index, year_cnt.values, alpha=0.3)
    ax2.set_ylabel("Listings count")
    plt.tight_layout(); plt.show()
else:
    print("Need 'year' and 'selling_price'.")


In [ ]:

# SECTION 10 — Top 10 Most Valuable Figures by Segment (condition × authenticity)
required = {"figure", "authenticity_n", "selling_price", "condition"}
if required.issubset(set(dfv.columns)):
    stats = (dfv.dropna(subset=["selling_price"])
               .groupby(["condition", "authenticity_n", "figure"], as_index=False)
               .agg(avg_price=("selling_price", "mean"),
                    median_price=("selling_price", "median"),
                    count=("selling_price", "count")))

    MIN_COUNT = 5
    TOP_N = 10
    stats = stats[stats["count"] >= MIN_COUNT]

    def topn_for(cond, auth, sort_by="median_price"):
        seg = stats[(stats["condition"] == cond) & (stats["authenticity_n"] == auth)].copy()
        if seg.empty: 
            return seg
        return seg.sort_values(sort_by, ascending=False).head(TOP_N)

    segments = [
        ("moc_figure", 1, "MOC & Graded"),
        ("moc_figure", 0, "MOC & Not Graded"),
        ("loose_figure", 1, "Loose & Graded"),
        ("loose_figure", 0, "Loose & Not Graded"),
    ]

    results = {}
    for cond, auth, label in segments:
        results[label] = topn_for(cond, auth)

    # Display
    for label, df_out in results.items():
        print(f"\n{label} — Top {TOP_N} (min {MIN_COUNT} sales, by median):")
        if df_out.empty:
            print("  (No data)")
        else:
            display(df_out[["figure", "avg_price", "median_price", "count"]].round(2))
else:
    print("Missing required columns for this section:", required - set(dfv.columns))


In [ ]:

# SECTION 11 — Top 10 Most Sold (overall & by segment)
if {"figure", "selling_price"}.issubset(dfv.columns):
    most_sold = (dfv.groupby("figure", as_index=False)
                   .agg(total_sales=("selling_price", "count"),
                        avg_price=("selling_price", "mean"))
                   .sort_values("total_sales", ascending=False)
                   .head(10))
    print("Top 10 most sold — overall:")
    display(most_sold.round(2))

if {"condition", "authenticity_n", "figure", "selling_price"}.issubset(dfv.columns):
    by_seg = (dfv.groupby(["condition", "authenticity_n", "figure"], as_index=False)
                .agg(total_sales=("selling_price", "count"),
                     avg_price=("selling_price", "mean"))
                .sort_values(["condition", "authenticity_n", "total_sales"], ascending=[True, True, False]))
    top10_by_seg = by_seg.groupby(["condition", "authenticity_n"]).head(10)
    print("\nTop 10 most sold — by segment:")
    display(top10_by_seg.round(2))


In [ ]:

# SECTION 12 — Pareto: Sales concentration by character_type
if {"character_type", "sales"}.issubset(dfv.columns):
    agg = dfv.groupby("character_type")["sales"].sum().sort_values(ascending=False)
    if len(agg):
        total = agg.sum() if agg.sum() else 1.0
        cshare = (agg / total).cumsum()
        topn = min(15, len(agg))

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.bar(agg.index[:topn], agg.values[:topn])
        ax.set_xticklabels(agg.index[:topn], rotation=45, ha="right")
        ax.set_ylabel("sales"); ax.set_xlabel("character_type")
        ax.set_title("Pareto: sales by character_type (bars) + cumulative share (line)")

        ax2 = ax.twinx()
        ax2.plot(cshare.index[:topn], cshare.values[:topn], marker="o", color="black")
        ax2.set_ylabel("cumulative share")
        plt.tight_layout(); plt.show()

        print(f"Top {topn} types account for {cshare.values[topn-1]*100:.1f}% of total sales.")
    else:
        print("No sales data after filters.")
else:
    print("Need 'character_type' and 'sales'.")



## SECTION 13 — Key Insights
- **Price distributions are right-skewed;** medians < means. Consider `USE_LOG=True` for robustness in comparative plots.
- **Grading (authenticity=1)** and **MoC condition** consistently raise prices; effect visible in KDEs and boxplots.
- **Top-valued figures** (by median) are stable across segments; using median reduces outlier bias vs mean.
- **Most sold figures** differ from most valuable ones — volume ≠ value.
- **Character_type concentration** can be substantial; top categories often account for a large share of sales.
